In [3]:
!pip install tqdm

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [4]:
!pip install tenacity

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [4]:
import os
import io
import json, glob
import urllib3
from PIL import *
import PIL.Image
import numpy as np
import pandas as pd
from tenacity import *

In [20]:
def parse_dataset(dataset, outdir, _max=10000):
    """
    parse the dataset to create a list of tuple containing absolute path and url of image
    :param _dataset: dataset to parse
    :param _outdir: output directory where data will be saved
    :param _max: maximum images to download (change to download all dataset)
    :return: list of tuple containing absolute path and url of image
    """
    _fnames_urls = []
    with open(dataset, 'r') as f:
        data = json.load(f)
        for image in data["images"]:
            url = image["url"]
            fname = os.path.join(outdir, "{}.jpg".format(image["imageId"]))
            _fnames_urls.append((fname, url))
    return _fnames_urls[:_max]
def download_image(fnames_and_urls):
    """
    download image and save its with 90% quality as JPG format
    skip image downloading if image already exists at given path
    :param fnames_and_urls: tuple containing absolute path and url of image
    """
    fname, url = fnames_and_urls
    if not os.path.exists(fname):
        http = urllib3.PoolManager(retries=Retry(connect=3, read=2, redirect=3))
        response = http.request("GET", url)
        image = Image.open(io.BytesIO(response.data))
        image_rgb = image.convert("RGB")
        image_rgb.save(fname, format='JPEG', quality=90)
        
%cd '/home/ec2-user/SageMaker/imat/'

parse = parse_dataset('test.json', '/home/ec2-user/SageMaker/imat/test_images',1000)

for i in parse:
    download_image(i)

/home/ec2-user/SageMaker/imat


In [6]:
test_json = json.load(open('/home/ec2-user/SageMaker/imat/test.json'))

In [7]:
num_images = len(test_json['images'])
print(num_images)

29809


In [37]:
# Get image ids
image_id = [int(test_json['images'][i]['imageId']) for i in range(num_images)]

In [38]:
image_id=image_id[0:1000]
len(image_id)

1000

In [39]:
def get_index(path):
    return int(path.split('/')[-1].split('.')[0])


In [40]:
# Get validation image paths
test_path_format = '/home/ec2-user/SageMaker/imat/test_images/*.jpg'
image_path = sorted(glob.glob(test_path_format), key = get_index)

In [41]:
len(image_path)

1000

In [42]:
test_csv = pd.DataFrame({'imageId': image_id, 'imagePath': image_path})

In [43]:
test_csv.head()

,imageId,imagePath
0,9898,/home/ec2-user/SageMaker/imat/test_images/9898...
1,9899,/home/ec2-user/SageMaker/imat/test_images/9899...
2,9900,/home/ec2-user/SageMaker/imat/test_images/9900...
3,9901,/home/ec2-user/SageMaker/imat/test_images/9901...
4,9902,/home/ec2-user/SageMaker/imat/test_images/9902...


In [44]:
test_csv.to_csv('test.csv', index=False)